In [ ]:
from IPython import get_ipython
import os

ipython = get_ipython()
current_dir = os.getcwd()

os.environ["GIT_DISCOVERY_ACROSS_FILESYSTEM"] = "1"
dir_name = "sapphire2"

# Define the repository path
if not current_dir.endswith(dir_name):
    repo_path = f"{current_dir}/{dir_name}"
else:
    repo_path = current_dir
    


# Check if the repository already exists
if os.path.exists(repo_path):
    # Use %cd as a magic command
    ipython.run_line_magic("cd", repo_path)
    !git fetch origin
else:
    # Clone the repository if it does not exist
    ipython.run_line_magic("cd",repo_path )
    !git clone https://github.com/DeepHansda/sapphire2.git
    ipython.run_line_magic("cd", repo_path)


In [ ]:
import subprocess
import threading
import time
import socket
import urllib.request

def run_subprocess(cmd):
    process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE )
    
    # Print output as it becomes available
    for line in process.stdout:
        if "http" in line.decode():
            link = line.decode().strip()
            print(f"This is the Link For Frontend, Click the Link and You can access the APP : {link}")
            break  # Stop after finding the link

            
        print(line, end='')
    
    # Print errors, if any
    for line in process.stderr:
        print(line, end='')


In [ ]:
!pip install aria2p tqdm diffusers peft fastapi uvicorn python-multipart pyngrok

!apt-get install aria2  -y  # Linux, or use Homebrew on Mac

# Start aria2c with RPC enabled in the background
!aria2c --enable-rpc --rpc-listen-all=true --daemon=true


In [ ]:
js_path = f"{repo_path}/src/js_modules"

# Change directory to js_modules using %cd
%cd {js_path}

# Install npm dependencies
!npm install -g localtunnel
!npm install -g pm2
!npm install


!npm run build
!pm2 start "npm run dev" --name tailwind-watch

In [ ]:
import os
from threading import Timer
from queue import Queue
from pyngrok import ngrok

# Step 2: Detect Environment and Retrieve Tokens
try:
    # Get the current working directory
    current_dir = os.getcwd()
    ngrok_token = input("Enter Ngrok Token Here:") or None
    if ngrok_token:
        print("Ngrok token already exists. Skipping retrieval process.")
    else:
        if "/content" in current_dir:  # Google Colab
            try:
                from google.colab import userdata
                ngrok_token = userdata.get("NGROK_TOKEN")
                print("Detected environment: Google Colab")
            except Exception as colab_error:
                print("Error retrieving tokens from Google Colab:", colab_error)

        elif "/kaggle" in current_dir:  # Kaggle
            try:
                from kaggle_secrets import UserSecretsClient
                user_secrets = UserSecretsClient()
                ngrok_token = user_secrets.get_secret("NGROK_TOKEN")
                print("Detected environment: Kaggle")
            except Exception as kaggle_error:
                print("Error retrieving tokens from Kaggle:", kaggle_error)

        else:  # Other environments
            ngrok_token = os.environ.get("NGROK_TOKEN")
            print("Detected environment: Other (Environment Variables)")

    # If ngrok_token is still None, raise an error
    if not ngrok_token:
        raise ValueError("Ngrok token is missing. Please set the 'NGROK_TOKEN'.")

except Exception as e:
    print("Error detecting environment or retrieving tokens:", e)


In [ ]:
def ngrok_tunnel(port, queue, auth_token):
    try:
        ngrok.set_auth_token(auth_token)
        url = ngrok.connect(port)
        queue.put(url)
    except Exception as e:
        print("Error starting ngrok tunnel:", e)

# **Start NGROK Tunnel :**

In [ ]:
src_path = f"{repo_path}/src"
%cd {src_path}

try:
    ngrok_output_queue = Queue()
    ngrok_thread = Timer(2, ngrok_tunnel, args=(8000, ngrok_output_queue, ngrok_token))
    ngrok_thread.start()
    ngrok_thread.join()
    print("Ngrok URL:", ngrok_output_queue.get())
except Exception as e:
    print("Error starting ngrok thread:", e)
    
    
!python server.py


# **Start Local Tunnel:**

In [ ]:
src_path = f"{repo_path}/src"
%cd {src_path}


port = 8000

def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
    print("\n trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")
    print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

    p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
    for line in p.stdout:
        print(line.decode(), end='')
        
thread = threading.Thread(target=iframe_thread,name="tunnel", args=(port,))
thread.start()
!python server.py